SOLID
Создайте приложение для эмуляции работа киоска
по продаже хот-догов. Приложение должно иметь следующую функциональность:
1. Пользователь может выбрать из трёх стандартных
рецептов хот-дога или создать свой рецепт.
2. Пользователь может выбирать добавлять ли майонез,
горчицу, кетчуп, топпинги (сладкий лук, халапеньо,
чили, соленный огурец и т.д.).
3. Информацию о заказанном хот-доге нужно отображать на экран и сохранять в файл.
4. Если пользователь заказывает от трёх хот-догов нужно
предусмотреть скидку.Скидка зависит от количества
хот-догов.
5. Расчет может производиться, как наличными, так и
картой.
6. Необходимо иметь возможность просмотреть количество проданных хот-догов, выручку, прибыль.
7. Необходимо иметь возможность просмотреть информацию о наличии компонентов для создания хот-дога.
8. Если компоненты для создания хот-догов заканчиваются нужно вывести информационное сообщение

In [ ]:
from abc import ABC, abstractmethod
from datetime import datetime

# Класс хот-дога и строителя 
class HotDog:
    def __init__(self, name):
        self.name = name
        self.toppings = []
        self.sauces = []
        self.base_price = 150  # Базовая цена хот-дога

    def add_topping(self, topping):
        self.toppings.append(topping)
        return self

    def add_sauce(self, sauce):
        self.sauces.append(sauce)
        return self

    def calculate_price(self):
        return self.base_price + len(self.toppings) * 20 + len(self.sauces) * 10

    def __str__(self):
        toppings_str = ', '.join(self.toppings) if self.toppings else "нет"
        sauces_str = ', '.join(self.sauces) if self.sauces else "нет"
        return f"{self.name} (Цена: {self.calculate_price()}₽)\nТоппинги: {toppings_str}\nСоусы: {sauces_str}"


class HotDogBuilder:
    def __init__(self, name):
        self.hotdog = HotDog(name)

    def add_topping(self, topping):
        self.hotdog.add_topping(topping)
        return self

    def add_sauce(self, sauce):
        self.hotdog.add_sauce(sauce)
        return self

    def build(self):
        return self.hotdog


class StandardHotDogs:
    @staticmethod
    def classic():
        return HotDogBuilder("Классический")\
            .add_sauce("кетчуп")\
            .add_topping("лук")\
            .build()

    @staticmethod
    def spicy():
        return HotDogBuilder("Острый")\
            .add_sauce("горчица")\
            .add_topping("халапеньо")\
            .add_topping("чили")\
            .build()

    @staticmethod
    def deluxe():
        return HotDogBuilder("Делюкс")\
            .add_sauce("кетчуп")\
            .add_sauce("майонез")\
            .add_topping("лук")\
            .add_topping("соленый огурец")\
            .add_topping("сладкий лук")\
            .build()

# Классы оплаты 
class PaymentStrategy(ABC):
    @abstractmethod
    def pay(self, amount):
        pass


class CashPayment(PaymentStrategy):
    def pay(self, amount):
        print(f"Оплачено {amount}₽ наличными")
        return True


class CardPayment(PaymentStrategy):
    def pay(self, amount):
        print(f"Оплачено {amount}₽ картой")
        return True

# Управление инвентарем 
class Inventory:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls._instance.ingredients = {
                "булка": 100, 
                "сосиска": 50, 
                "лук": 30, 
                "халапеньо": 20, 
                "чили": 15,
                "соленый огурец": 25,
                "сладкий лук": 20,
                "кетчуп": 50, 
                "горчица": 50,
                "майонез": 40
            }
        return cls._instance

    def check_stock(self, ingredient):
        return self.ingredients.get(ingredient, 0)

    def update_stock(self, ingredient, quantity):
        if ingredient in self.ingredients:
            self.ingredients[ingredient] -= quantity
            if self.ingredients[ingredient] <= 5:
                print(f"⚠️ Внимание! Заканчивается: {ingredient} (осталось: {self.ingredients[ingredient]})")
            return True
        return False

    def get_low_stock_items(self):
        return {item: qty for item, qty in self.ingredients.items() if qty <= 5}

    def display_stock(self):
        print("\nТекущие запасы:")
        for item, qty in self.ingredients.items():
            print(f"{item}: {qty}")

#  Основной класс киоска 
class Kiosk:
    def __init__(self):
        self.inventory = Inventory()
        self.orders = []
        self.revenue = 0
        self.profit = 0
        self.total_sold = 0

    def make_order(self, hotdog, quantity=1):
        required_ingredients = ["булка", "сосиска"] + hotdog.toppings + hotdog.sauces
        
        for ingredient in required_ingredients:
            if self.inventory.check_stock(ingredient) < quantity:
                print(f"Недостаточно ингредиента: {ingredient}")
                return False
        
        for ingredient in required_ingredients:
            self.inventory.update_stock(ingredient, quantity)
        
        self.orders.extend([hotdog] * quantity)
        print(f"\nДобавлен заказ: {hotdog.name} x{quantity}")
        return True

    def apply_discount(self, total):
        if len(self.orders) >= 3:
            discount = min(20, len(self.orders))
            discounted = total * (1 - discount / 100)
            print(f"Применена скидка {discount}%")
            return discounted
        return total

    def calculate_order_cost(self):
        total = sum(hotdog.calculate_price() for hotdog in self.orders)
        return self.apply_discount(total)

    def checkout(self, payment_strategy):
        if not self.orders:
            print("Нет заказов для оплаты")
            return False
        
        total = self.calculate_order_cost()
        if payment_strategy.pay(total):
            self.revenue += total
            self.profit += total * 0.4
            self.total_sold += len(self.orders)
            self._save_to_file()
            self.orders = []
            return True
        return False

    def _save_to_file(self):
        with open("orders.txt", "a", encoding="utf-8") as f:
            f.write(f"\n=== Заказ от {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} ===\n")
            for order in self.orders:
                f.write(str(order) + "\n")
            f.write(f"Итого: {self.calculate_order_cost()}₽\n")

    def display_stats(self):
        print("\nСтатистика киоска:")
        print(f"Всего продано хот-догов: {self.total_sold}")
        print(f"Общая выручка: {self.revenue}₽")
        print(f"Прибыль: {self.profit:.2f}₽")

    def cancel_order(self):
        if self.orders:
            print("Текущий заказ отменен")
            self.orders = []
        else:
            print("Нет активных заказов для отмены")

# Пользовательский код
def display_menu():
    print("\n" + "="*50)
    print("МЕНЮ КИОСКА ХОТ-ДОГОВ".center(50))
    print("="*50)
    print("\nСтандартные хот-доги:")
    print("1. Классический")
    print("2. Острый")
    print("3. Делюкс")
    print("4. Создать свой хот-дог")
    print("\nДополнительные опции:")
    print("5. Посмотреть статистику")
    print("6. Проверить запасы ингредиентов")
    print("7. Оплатить заказ")
    print("8. Отменить заказ")
    print("0. Выход")
    print("="*50)

def create_custom_hotdog():
    builder = HotDogBuilder("Кастомный")
    
    print("\nДоступные топпинги:")
    toppings = ["лук", "халапеньо", "чили", "соленый огурец", "сладкий лук"]
    for i, topping in enumerate(toppings, 1):
        print(f"{i}. {topping}")
    
    print("\nДоступные соусы:")
    sauces = ["кетчуп", "горчица", "майонез"]
    for i, sauce in enumerate(sauces, 1):
        print(f"{i}. {sauce}")
    
    print("\nВыберите топпинги (через пробел, 0 - пропустить):")
    choices = input().split()
    for choice in choices:
        if choice.isdigit() and 0 < int(choice) <= len(toppings):
            builder.add_topping(toppings[int(choice)-1])
    
    print("\nВыберите соусы (через пробел, 0 - пропустить):")
    choices = input().split()
    for choice in choices:
        if choice.isdigit() and 0 < int(choice) <= len(sauces):
            builder.add_sauce(sauces[int(choice)-1])
    
    return builder.build()

def main():
    kiosk = Kiosk()
    inventory = Inventory()
    
    while True:
        display_menu()
        choice = input("\nВыберите опцию: ")
        
        if choice == "1":
            kiosk.make_order(StandardHotDogs.classic())
        elif choice == "2":
            kiosk.make_order(StandardHotDogs.spicy())
        elif choice == "3":
            kiosk.make_order(StandardHotDogs.deluxe())
        elif choice == "4":
            custom_hotdog = create_custom_hotdog()
            print("\nВаш хот-дог:")
            print(custom_hotdog)
            quantity = int(input("Введите количество: "))
            kiosk.make_order(custom_hotdog, quantity)
        elif choice == "5":
            kiosk.display_stats()
        elif choice == "6":
            inventory.display_stock()
            low_stock = inventory.get_low_stock_items()
            if low_stock:
                print("\n Заканчиваются следующие ингредиенты:")
                for item, qty in low_stock.items():
                    print(f"{item}: осталось {qty}")
        elif choice == "7":
            print("\nСпособы оплаты:")
            print("1. Наличные")
            print("2. Карта")
            payment_choice = input("Выберите способ оплаты: ")
            if payment_choice == "1":
                kiosk.checkout(CashPayment())
            elif payment_choice == "2":
                kiosk.checkout(CardPayment())
            else:
                print("Неверный выбор")
        elif choice == "8":
            kiosk.cancel_order()
        elif choice == "0":
            print("До свидания!")
            break
        else:
            print("Неверный выбор, попробуйте снова")

if __name__ == "__main__":
    main()


              МЕНЮ КИОСКА ХОТ-ДОГОВ               

Стандартные хот-доги:
1. Классический
2. Острый
3. Делюкс
4. Создать свой хот-дог

Дополнительные опции:
5. Посмотреть статистику
6. Проверить запасы ингредиентов
7. Оплатить заказ
8. Отменить заказ
0. Выход

Текущие запасы:
булка: 100
сосиска: 50
лук: 30
халапеньо: 20
чили: 15
соленый огурец: 25
сладкий лук: 20
кетчуп: 50
горчица: 50
майонез: 40

              МЕНЮ КИОСКА ХОТ-ДОГОВ               

Стандартные хот-доги:
1. Классический
2. Острый
3. Делюкс
4. Создать свой хот-дог

Дополнительные опции:
5. Посмотреть статистику
6. Проверить запасы ингредиентов
7. Оплатить заказ
8. Отменить заказ
0. Выход
До свидания!
